## Toliman Pupil Characterisation ✨

This notebook calculates the phase errors on the Toliman pupil.

Existing system aberrations are loaded and can be calculated using System_Aberrations.ipynb

In [ ]:
import dLux as dl
import dLux.utils as dlu

import jax.numpy as jnp
import numpy as np
import jax.scipy as jsp

import zodiax as zdx
import optax
from tqdm.notebook import tqdm

from skimage.io import imread
from skimage.filters import window
import skimage as ski
from skimage.transform import resize

import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
